In [1]:
import requests
import json
from pprint import pprint
import pandas as pd
from tabulate import tabulate
import pprint

# Setup

In [8]:
Baseurl = 'https://kgsa-dev.kochcloud.com'
##Baseurl = 'https://kgsa-dev.kochcloud.com'
user = 'badrul'
password = "%IaY0lolDEOeQqsii$w9UO"

# Model Training

### C3 FHR Data

#### Data Format
1. **First column** - should be a datetime column

In [9]:
## TODO: Read in the data and show the head
s3DataPath = "s3://prediction-services/data/TrainingInputData_Transformed_Test_Sample_2.csv"
fhrDf = pd.read_csv(s3DataPath, sep=',', header = 0, index_col = 0)
fhrDf.index = pd.to_datetime(fhrDf.index)
##fhrDf.head()

In [10]:
print(tabulate(fhrDf, headers='keys', tablefmt='psql'))


+---------------------+------------------+--------------+---------------+--------------+--------------+--------------+--------------+
| Date                |       19463-3243 |   34883-3350 |   53511-10483 |   53511-3199 |   53511-3553 |   53511-3802 |   53511-4773 |
|---------------------+------------------+--------------+---------------+--------------+--------------+--------------+--------------|
| 2017-12-01 00:00:00 |      0           |        46480 |             0 |            0 |            0 |            0 |            0 |
| 2018-01-01 00:00:00 |      0           |            0 |             0 |            0 |            0 |            0 |            0 |
| 2018-02-01 00:00:00 |      0           |       144000 |         42000 |            0 |        42000 |            0 |        37500 |
| 2018-03-01 00:00:00 |      1.49072e+06 |       240000 |        168000 |       214500 |        21000 |       319500 |       306000 |
| 2018-04-01 00:00:00 |      1.19184e+06 |       732180 |     

### Training Parameters
Dictionary below contains the minimum configuration needed to run a training job.

In [11]:
trainingDict = {
        "training": {
            "train_name": "model-minimum-config-jupyter", 
            "target_data_location": "s3://prediction-services/data/TrainingInputData_Transformed_Test_Sample_2.csv",
            "train_data_end_dtm": "2020-06-01",
            "test_data_end_dtm": "2020-09-01",
            "validation_data_end_dtm": "2020-12-01",
            "model_names": ["ARIMA", "HOLTWINTERS", "PROPHET"] 
        }
    }


#### Training Configuration Explanation


1. **train_name** = Used to identify training job and used in model version
2. **target_data_location** = S3 (only S3 is supported) location of the CSV file containing the training data
3. **train_data_end_dtm** - Last inclusive day to be used for training
4. **test_data_end_dtm** - Last inclusive day to be used for testing
5. **validation_data_end_dtm** - Last day or the remaining data
6. **model_names** - List of models to be trained on the data


#### Important Default Parameters
1. Default training task is **Model** which trains on the data using single default set of model parameters
2. Code will run in **Sequential** mode, i.e., on a single core
3. Default loss function is **MAPE**
4. Default prediction frequency is **Monthly** 

### Executing Training Job

In [12]:
response = requests.post(Baseurl+"/training", 
                         auth=(user, password),
                         #data = json.dumps(trainingDict))  
                         json = trainingDict)

Training can be divided into two categories:
1. **Model** - Training one or more models based on single parameter set (either default or user provided)
2. **Tuning** - Train one or more models in automated mode (population based training) or configured supported algorithms.  Only the following algorithms are supported:
    1. Grid search: *GRID-SEARCH*
    2. Random search: *RANDOM*
    3. Bayesian Optimization: *BAYESIAN*
    4. Population Based Training: *PBT*
    
To train or tune use the base url + /training
Upon the successful submission of the training to the Prediction Service, training ID and status will be returned.
Training ID is needed to get status and results of training.

### Checking Training Status

In [13]:
print(response)
result = response.json()
print(pprint.pprint(result))
trainingId = result['runId']
print(trainingId)

<Response [200]>
{'runId': '3f8f1586-2be3-4b39-9b74-b897115e8df6', 'status': 'Submitted'}
None
3f8f1586-2be3-4b39-9b74-b897115e8df6


#### Result of Calling the Training Job

The training is executed asynchronously.  This end point submits the job to the prediction service which gets queued and run as resource become available.  Thus, the end point returns a **runId** which is needed to get status, training results or use the trained model for prediction

In [8]:
status_response = requests.get(Baseurl+"/trainings/" + trainingId,
                              auth=(user,password))    
print(status_response)

<Response [200]>


In [10]:
trainingStatus = status_response.json()
print(pprint.pprint(trainingStatus))

{'runId': '4690e6e9-2f1a-4912-9816-59b27ad8cc26',
 'status': 'Running',
 'trainStartTs': '2021-02-17T15:01:07.467155',
 'updateTs': '2021-02-17T15:01:07.413007'}
None


#### Parameters Returned by Status Call

1. **runId** - This is an UUID that uniquely identifies each training job.  This UUID is **very important** as all subsequent information and result of training is identified using the UUID.  The UUID can also be used during prediction to use the model trained
2. **status**: Current status of the training
3. **trainStartTs**: Timestamp of the training start 

### Extracting Training Results

In [23]:
resultResponse = requests.get(Baseurl+"/trainings/"+ trainingId,
                              auth=(user,password))    
print(resultResponse)
trainResults = resultResponse.json()
print(pprint.pprint(trainResults))

<Response [200]>
{'resultLocation': 's3://prediction-services/train/model-minimum-config-jupyter_a7ca04f5-4df6-4a91-b432-acf71a56ffe1_2021-02-17-03.51.37.csv',
 'runId': 'a7ca04f5-4df6-4a91-b432-acf71a56ffe1',
 'status': 'Completed',
 'trainEndTs': '2021-02-17T03:52:15.726194',
 'trainStartTs': '2021-02-17T03:51:37.107267',
 'updateTs': '2021-02-17T03:51:37.017906'}
None


#### Training Result Information

Paramters Explnation:
1. **status** - Status of the training job.
2. **resultLocation** - S3 location where training scores will be stored once the training has completed


In [24]:
##TODO: Read in the forecast results and show 
trainResultLoc = trainResults['resultLocation']
##trainResultLoc = 's3://prediction-services/train/model-minimum-config-jupyter_d1fe11ae-27dc-44ca-93aa-9ba367786872_2021-01-29-20.52.29.csv'
trainDf = pd.read_csv(trainResultLoc, sep=',', header = 0)
trainDf.head()
##fhrDf.index = pd.to_datetime(fhrDf.index)
##print(tabulate(trainDf, headers='keys', tablefmt='psql'))

,target_column,train_start_ts,train_end_ts,model,model_params,train_score,test_score,validation_score
0,53511-10483,2021-02-17 03:51:37.069228,2021-02-17 03:51:53.612120,PROPHET,"{'parameters': {'growth': 'linear', 'holidays'...",0.068470,0.154862,1.000000
1,53511-3553,2021-02-17 03:51:37.069228,2021-02-17 03:52:04.359351,PROPHET,"{'parameters': {'growth': 'linear', 'holidays'...",1.000000,1.000000,1.000000
2,53511-3802,2021-02-17 03:51:37.069228,2021-02-17 03:52:09.724905,PROPHET,"{'parameters': {'growth': 'linear', 'holidays'...",0.775224,0.611006,0.659785
3,53511-4773,2021-02-17 03:51:37.069228,2021-02-17 03:52:15.311753,PROPHET,"{'parameters': {'growth': 'linear', 'holidays'...",0.249423,0.247317,1.000000
4,34883-3350,2021-02-17 03:51:37.069228,2021-02-17 03:51:48.232401,PROPHET,"{'parameters': {'growth': 'linear', 'holidays'...",0.685172,1.000000,0.259096


# Model Tuning and Experimentation

#### Hyperparameter Tuning Introduction
**Tuning jobs scale by**:
1. Scaling up or Vertical Scaling - Parallel or
2. Scaling out or Horizontal Scaling - Distributed

Univariate models execute independently.  For each server number of models being tuned simultaneously is limited by number of CPUs on the server

**Deep Learning Models and Parameter Server**
A parameter server typically exists as a remote process or service and interacts with clients through remote procedure calls.

Image(filename=images/param-server-arch.jpg)

![Parameter Server Architecture](images/param-server-arch.jpg)

For hyperparameter tuning a search space of paerameters has to be defined.

##### Search Algorithms
Search Algorithms are wrappers around open-source optimization libraries for efficient hyperparameter selection.

Search Algorithms cannot affect or stop training processes. However, you can use them together to early stop the evaluation of bad trials.

1. **Bayesian Optimization**: This constrained global optimization process builds upon bayesian inference and gaussian processes. It attempts to find the maximum value of an unknown function in as few iterations as possible. This is a good technique for optimization of high cost functions.
2. **BOHB (Bayesian Optimization HyperBand)**: An algorithm that both terminates bad trials and also uses Bayesian Optimization to improve the hyperparameter search. It is backed by the HpBandSter library. BOHB is intended to be paired with a specific scheduler class: HyperBandForBOHB.
3. **HyperOpt**: A Python library for serial and parallel optimization over awkward search spaces, which may include real-valued, discrete, and conditional dimensions.
4. **Scikit-Optimize*:
5. **Nevergrad**: HPO without computing gradients.


##### Schedulers
Schedulers are distributed implementations of early-stopping algorithm
Schedulers can early terminate bad trials, pause trials, clone trials, and alter hyperparameters of a running trial.

1. **Median Stopping Rule**: It applies the simple rule that a trial is aborted if the results are trending below the median of the previous trials.
2. **Population Based Training (PBT)** - 
3. **FIFOScheduler* - Simple scheduler that just runs trials in submission order

#### Experiment Execution

### Tuning Parameters
Dictionary below contains the minimum configuration needed to run a tuning job.

In [5]:
trainingDict = {
        "training": {
            "train_task_type": "TUNING",
            "train_name": "TUNING-arima-config-jupyter", 
            "target_data_location": "s3://prediction-services/data/h0500hn_ft-worth.csv",
            "train_data_end_dtm": "2019-12-01",  
            "test_data_end_dtm": "2020-06-01",         
            "validation_data_end_dtm": "2020-11-01",
            "model_names": ["ARIMA"] 
        },
        "models": {
            "ARIMA": {
            "model_name": "ARIMA",
            "model_time_interval": "M",
            "hyperparam_alg": "GRID-SEARCH", #RANDOM, BAYESIAN
            "model_config": {
                "parameters": {"p": [1, 2,3,4,5,6,7], "d": [1, 2], "q": [0, 1,2]},
                "hyperparameters": {"disp": 0}}
            }
        }
    }
response = requests.post(Baseurl+"/training", 
                         auth=(user, password),
                         data = json.dumps(trainingDict))  
                         #json = trainingDict)


#### Tuning Configuration Explanation
##### Required Parameters
1. **train_task_type** = "TUNING"
    * By default it's set to "MODEL"
2. **All the parameters mentioned in the training section**

##### Optional Parameters
1. **hyperparameter_algorithm** -


In [16]:
print(response)
result = response.json()
print(pprint.pprint(result))
trainingId = result['runId']
print(trainingId)


<Response [422]>
{'detail': [{'ctx': {'given': 'TUNING', 'permitted': ['Model', 'Tuning']},
             'loc': ['body', 'training', 'train_task_type'],
             'msg': "unexpected value; permitted: 'Model', 'Tuning'",
             'type': 'value_error.const'}]}
None


KeyError: 'runId'

In [5]:
resultResponse = requests.get(Baseurl+"/trainings/"+ trainingId,
                              auth=(user,password))    
print(resultResponse)
trainResults = resultResponse.json()
print(pprint.pprint(trainResults))

<Response [200]>
{'Status': 'Running', 'UpdateTs': 'Tue, 12 Jan 2021 02:17:57 GMT', 'resultLocation': 's3://prediction-services/train/', 'runId': '6b320ef3-917f-4ebf-9b7c-d0fc319ebfdb', 'trainEndTs': None, 'trainStartTs': 'Tue, 12 Jan 2021 02:17:57 GMT'}


# Prediction or Scoring

### Scoring Parameters
Dictionary below contains the minimum configuration needed to run a scoring job.

In [11]:
scoringDict = {
        "scoring": {
            "score_name": "score-minimum-config-manager",
            "target_data_location": "s3://prediction-services/data/TrainingInputData_Transformed_Test_Sample_2.csv",
            "model_names": ["ARIMA", "HOLTWINTERS"],
            "prediction_steps": 12,
            "prediction_count": 10
            
        }
    }
predResponse = requests.post(Baseurl+ "/predictions", 
                         auth=(user, password),
                         data = json.dumps(scoringDict))  

### Scoring Configuration Explanation
#### Required Parameters
1. **score_name** - 
2. **target_data_location** -
3. **model_names** -

#### Optional Parameters
1. **prediction_steps** - 
2. **prediction_count** - 

In [12]:
print(predResponse)
predResults = predResponse.json()
##print(predResults)
print(pprint.pprint(predResults))
predId = predResults['runId']
print(predId)

<Response [200]>
{'runId': '29788f02-6acf-4808-945a-71ea43d5af6a', 'status': 'Created'}
None
29788f02-6acf-4808-945a-71ea43d5af6a


### Get Prediction Status

In [26]:
predResponse = requests.get(Baseurl+"/predictions/"+ predId,
                              auth=(user,password))    
print(predResponse)
predStatus = predResponse.json()
print(pprint.pprint(predStatus))
predId = predStatus['runId']
print(predId)

<Response [200]>
{'runId': '8e680703-a618-4ea6-bcce-a27f03b6aecc', 'status': 'Completed'}
None
8e680703-a618-4ea6-bcce-a27f03b6aecc


### Fetch Prediction Results

In [13]:
predResponse = requests.get(Baseurl+"/predictions/"+ predId,
                              auth=(user,password))    
print(predResponse)
predResults = predResponse.json()
##print(predResults)
print(pprint.pprint(predResults))
predLoc = predResults["resultLocation"]
print(predLoc)

<Response [200]>
{'resultLocation': 's3://prediction-services/score/score-minimum-config-manager_29788f02-6acf-4808-945a-71ea43d5af6a_2021-02-17-15.02.59.csv',
 'runId': '29788f02-6acf-4808-945a-71ea43d5af6a',
 'scoreEndTs': '2021-02-17T15:03:00.727917',
 'scoreStartTs': '2021-02-17T15:02:59.802333',
 'status': 'Completed',
 'updateTs': '2021-02-17T15:02:59.772462'}
None
s3://prediction-services/score/score-minimum-config-manager_29788f02-6acf-4808-945a-71ea43d5af6a_2021-02-17-15.02.59.csv


### Forecast

In [14]:
predDf = pd.read_csv(predLoc, sep=',', header = 0)
predDf.head()
##fhrDf.index = pd.to_datetime(fhrDf.index)
##print(tabulate(trainDf, headers='keys', tablefmt='psql'))

,forecast_date,34883-3350,53511-10483,53511-3199,53511-3553,53511-3802,19463-3243,53511-4773
0,2021-04-01,627278.344957,16718.817007,67147.236808,7936.673735,45283.528897,164020.066938,36039.692221
1,2021-05-01,627718.069081,16422.009309,71096.830707,8664.854207,46432.778264,156884.896005,34939.179289
2,2021-06-01,628107.962759,16770.018017,72731.297351,8637.896984,47561.653915,163917.820953,35779.524294
3,2021-07-01,628453.679674,16897.878396,75242.287270,8981.081624,48699.565052,164180.676417,35916.965989
4,2021-08-01,628760.230593,17100.900148,77421.419384,9142.835778,49833.469065,167678.535456,36308.971942


# Prediction: Using Previously Trained Model

In [28]:
scoringDict = {
        "scoring": {
            "score_name": "score-minimum-config-manager",
            "target_data_location": "s3://prediction_services/data/TrainingInputData_Transformed_Test_Sample_2.csv",
            "model_names": ["ARIMA"],
            "prediction_steps": 12,
            "prediction_count": 10,
            "train_run_id": "e5ff99ac-e260-49d0-934f-46c46d31f136"
        }
    }
predResponse = requests.post(Baseurl+ "/predictions", 
                         auth=(user, password),
                         data = json.dumps(scoringDict))  

In [29]:
print(predResponse)
predStatus = predResponse.json()
print(pprint.pprint(predStatus))
predId = predStatus['runId']
print(predId)

<Response [200]>
{'runId': '20fcd3ab-9f30-4e52-9340-8b48981f1956', 'status': 'Created'}
None
20fcd3ab-9f30-4e52-9340-8b48981f1956


### Get Prediction Status

In [31]:
predResponse = requests.get(Baseurl+"/predictions/"+ predId,
                              auth=(user,password))    
print(predResponse)
predStatus = predResponse.json()
print(pprint.pprint(predStatus))
predId = predStatus['runId']
print(predId)

<Response [200]>
{'runId': '20fcd3ab-9f30-4e52-9340-8b48981f1956',
 'status': 'Failed',
 'status_message': 'Failed to write to S3'}
None
20fcd3ab-9f30-4e52-9340-8b48981f1956


### Fetch Prediction Results

In [32]:
predResponse = requests.get(Baseurl+"/predictions/"+ predId,
                              auth=(user,password))    
print(predResponse)
predResults = predResponse.json()
print(pprint.pprint(predResults))

<Response [200]>
{'resultLocation': 's3://prediction-services/score/',
 'runId': '20fcd3ab-9f30-4e52-9340-8b48981f1956',
 'scoreEndTs': '2021-02-17T03:57:54.286754',
 'scoreStartTs': '2021-02-17T03:57:54.109345',
 'status': 'Failed',
 'status_message': 'Failed to write to S3',
 'updateTs': '2021-02-17T03:57:54.028922'}
None
